In [1]:
import tkinter 
from tkinter import *
from tkinter import filedialog 
# import dask.dataframe as dd
import pandas as pd


In [ ]:
def upload_file():
    file=filedialog.askopenfilename(title="Select file" , filetypes=[("All files", '*.*'),("CSV files", '.csv')])
    # file=filedialog.askopenfilename(title="Select file" , filetypes=[("CSV files", '.csv')])

    if(file):
        lbl_my_file=Label(window, text="File read!", font=("arial", 12, "bold") )
        lbl_my_file.pack()
#     through dask
#         df=dd.read_csv(file)
#         file_name.set(file)
#         # fobj=open(file,'r')
#         file_data.set(df.compute())

# through pandas
        
        
        df=pd.read_csv(file)
        file_name.set(file)
#         print(df)
        df["name"] = df['name'].str.replace(" ", "")
        std_att = df['name'].astype(str) + df['age'].astype(str) + df['age_unit'].astype(str) + df['province_id'].astype(str) + df['district_id'].astype(str) + df['municipality_id'].astype(str) + df['tole'].astype(str) + df['ward'].astype(str) + df['caste'].astype(str) + df['sex'].astype(str)
#         print (std_attribute)
        df['std_attribute'] = std_att
        sorted_df = df.sort_values(by=['std_attribute'])
        print(sorted_df)


        

        

        
        
    
window=Tk()
window.geometry("400x300")
window.title("Data Redundancy Detection System")
title_lbl=Label(window, text="Upload .csv file", font=("arial", 30, "italic bold"),bd=7)
title_lbl.pack()
button=Button(window, text="Upload File", relief=RAISED,font=("arial", 15, "bold"), width=15, bg="light blue" ,command=upload_file)
button.pack()


file_name=StringVar()
file_data=StringVar()
my_file=StringVar()



lbl_name=Label(window, textvariable=file_name)
lbl_name.pack()



# lbl_data=Label(window, textvariable=file_data, relief=GROOVE)
# lbl_data.grid(row=4, columnspan=2 , padx=50, pady=5)


window.mainloop()

# lbl_data.pack(ipadx=10, ipady=10)



        id                                 token                 name  age  \
329    601  8b832dd1-20ae-43f7-9aea-d8f945be6a02         AakritiDhami    5   
866   2680  8ac6333c-8d5c-4fd3-96fa-5ba979666903       AalokChaudhary   20   
455    754  cd353936-f5f0-45b0-8dce-e0f37a8ef8c7      AaratiChaudhary   27   
864   2678  89611638-9d29-4205-8889-168a5c6a956b       AayushKumarSah   14   
81     251  c80933ac-2520-4966-8ab4-298086c330f8         AayushaShahi    1   
...    ...                                   ...                  ...  ...   
1098  3038  eddd4fd6-c879-40e0-b5f6-3b9390a89459      syamKishorYadav   30   
173    405  65c4d147-4e7b-414f-90f2-391c366a7bd7         umeshSarbuja   44   
728   1933      fd6f19c64d6dfe9e592118b54a48b352          urmilakarki   53   
887   2757      2bbbd6ee36c5331dccc88fce60a1424f     vhilashaAdhikari   22   
735   1941      c962faa60ff428b7502c00c481bb1036  yagyanidhikhatiwada   30   

      age_unit  province_id  district_id  municipality_id org_c